In [2]:
import csv
import numpy as np
import networkx as nx
import math
from scipy.stats import mode
import function
import pickle

## 得到用户使用过的点

In [3]:
# 每个用户使用过的点

def get_user_used_node():

    user_used_node = {}

    i = 0
    csv_reader = csv.reader(open('./data/train_hard.csv', encoding='utf-8'))
    for row in csv_reader:
        i+=1
        if i==1:
            continue
        user = row[1]
        start = row[5]
        end = row[6]
        man_dis = int(row[9])

      
        if user not in user_used_node:
            user_used_node[user] = set()

        user_used_node[user].add(start)
        user_used_node[user].add(end)

    i = 0
    csv_reader = csv.reader(open('./data/test_hard.csv', encoding='utf-8'))
    for row in csv_reader:
        i+=1
        if i==1:
            continue
        user = row[1]
        start = row[5]

        if user not in user_used_node:
            user_used_node[user] = set()

        user_used_node[user].add(start)
        
    return user_used_node


## 得到起点到了哪些终点

In [4]:
def get_node_top_destination(path):

    node_top_destination = {}

    i = 0
    csv_reader = csv.reader(open(path, encoding='utf-8'))
    for row in csv_reader:
        i+=1
        if i==1:
            continue
        
        start = row[5]
        end = row[6]
        
        if start not in node_top_destination:
            node_top_destination[start] = {}

        
        node_top_destination[start][end] = node_top_destination[start].get(end,0)+1

    for start in node_top_destination:
        tmp = sorted(node_top_destination[start].items(),key = lambda item:item[1],reverse=True)
        node_top_destination[start] = tmp
    
    return node_top_destination



In [46]:
# 计算覆盖率
def computer_coverage(user_used_node,node_top_destination,top_num,hour_node_top_num,hour_top_num):
       
    miss = 0# 
    
    i = 0
    csv_reader = csv.reader(open('./data/test_hard.csv', encoding='utf-8'))
    for row in csv_reader:
        i+=1
        if i==1:
            continue
        user = row[1]
        start = row[5]
        end = row[6]
        hour = row[7]
        
        candidates = set()
        
        temp = user_used_node.get(user,set())
        for node in temp:
            candidates.add(node)

        temp = node_top_destination.get(start,'')
        for k in range(min(top_num,len(temp))):
            candidates.add(temp[k][0])
            
        for hour in hour_node_top_num:
            temp = hour_node_top_num[hour].get(start,'')
            for k in range(min(hour_top_num,len(temp))):
                candidates.add(temp[k][0])
            
        if end not in candidates:
            miss+=1
            
    print(1-miss/(i-1))

In [31]:
def get_hour_node_top_num(path):
    
    hour_node_top_num = {}
    
    i = 0
    csv_reader = csv.reader(open(path, encoding='utf-8'))
    for row in csv_reader:
        i+=1
        if i==1:
            continue
        start = row[5]
        end = row[6]
        hour = row[7]
        
        if hour not in hour_node_top_num:
            hour_node_top_num[hour] = {}
        
        if start not in hour_node_top_num[hour]:
            hour_node_top_num[hour][start] = {}
        
        hour_node_top_num[hour][start][end] = hour_node_top_num[hour][start].get(end,0)+1
        
    for hour in hour_node_top_num:
        for start in hour_node_top_num[hour]:
            tmp = sorted(hour_node_top_num[hour][start].items(),key = lambda item:item[1],reverse=True)
            hour_node_top_num[hour][start] = tmp
    return hour_node_top_num

In [71]:
hour_node_top_num = get_hour_node_top_num('./data/train_end_geofly.csv')

In [42]:
user_used_node = get_user_used_node()

node_top_destination = get_node_top_destination('./data/train_hard.csv')

## node_top_destination的平均候选点

In [75]:
def get_mean_node_top_destination(node_top_destination):
    
    a = []
    
    start_to_end = {}
    for top_num in [3,5,10,20,30,40,50,500000]:
        
        for start in node_top_destination:
            if start not in start_to_end:
                start_to_end[start] = set()

            temp = node_top_destination.get(start,'')
            for k in range(min(top_num,len(temp))):
                start_to_end[start].add(temp[k][0])
        total = 0
        for start in start_to_end:
            total += len(start_to_end[start])
        a.append(total/len(start_to_end))
        print(top_num,total,len(start_to_end),total/len(start_to_end))
    for row in a:
        print(row)

In [76]:
node_top_destination = get_node_top_destination('./data/train_hard.csv')
get_mean_node_top_destination(node_top_destination)

node_top_destination = get_node_top_destination('./data/train_start_geofly.csv')
get_mean_node_top_destination(node_top_destination)

node_top_destination = get_node_top_destination('./data/train_end_geofly.csv')
get_mean_node_top_destination(node_top_destination)


3 211792 87270 2.4268591726824797
5 309285 87270 3.54400137504297
10 496767 87270 5.69229975936748
20 746242 87270 8.550956800733356
30 901734 87270 10.332691646613958
40 1001119 87270 11.471513693136243
50 1065576 87270 12.210106565830182
500000 1191278 87270 13.650486994385242
2.4268591726824797
3.54400137504297
5.69229975936748
8.550956800733356
10.332691646613958
11.471513693136243
12.210106565830182
13.650486994385242
3 416732 163643 2.546592277090985
5 630238 163643 3.8512982529041877
10 1077091 163643 6.581955842901927
20 1778938 163643 10.870846904542205
30 2342283 163643 14.313371179946591
40 2819520 163643 17.22970123989416
50 3232942 163643 19.756066559522864
500000 5698928 163643 34.82536986000012
2.546592277090985
3.8512982529041877
6.581955842901927
10.870846904542205
14.313371179946591
17.22970123989416
19.756066559522864
34.82536986000012
3 261810 87270 3.0
5 436350 87270 5.0
10 852569 87270 9.769325083075513
20 1490845 87270 17.08313280623353
30 2021626 87270 23.165188

## hour_node_top_num的平均候选点

In [72]:
def get_mean_hour_node_top_num(hour_node_top_num):
    
    a = []
    
    start_to_end = {}
    for hour_top_num in [3,5,10,20,30,40,50,500000]:
        for hour in hour_node_top_num:
            for start in hour_node_top_num[hour]:
                if start not in start_to_end:
                    start_to_end[start] = set()

                temp = hour_node_top_num[hour].get(start,'')
                for k in range(min(hour_top_num,len(temp))):
                    start_to_end[start].add(temp[k][0])
        total = 0
        for start in start_to_end:
            total += len(start_to_end[start])
        a.append(total/len(start_to_end))
        print(hour_top_num,total,len(start_to_end),total/len(start_to_end))
    for row in a:
        print(row)

In [73]:
hour_node_top_num = get_hour_node_top_num('./data/train_hard.csv')
get_mean_hour_node_top_num(hour_node_top_num)

3 1401664 87270 16.0612352469348
5 2159151 87270 24.741045032657272
10 3601746 87270 41.271295977999316
20 4683736 87270 53.66948550475536
30 5190772 87270 59.47945456628853
40 5464698 87270 62.618288071502235
50 5626006 87270 64.46666666666667
500000 5927657 87270 67.9231923914289
16.0612352469348
24.741045032657272
41.271295977999316
53.66948550475536
59.47945456628853
62.618288071502235
64.46666666666667
67.9231923914289


In [51]:
node_top_destination = {}
computer_coverage(user_used_node,node_top_destination,3,hour_node_top_num,3)
computer_coverage(user_used_node,node_top_destination,5,hour_node_top_num,5)
computer_coverage(user_used_node,node_top_destination,10,hour_node_top_num,10)
computer_coverage(user_used_node,node_top_destination,20,hour_node_top_num,20)
computer_coverage(user_used_node,node_top_destination,30,hour_node_top_num,30)
computer_coverage(user_used_node,node_top_destination,40,hour_node_top_num,40)
computer_coverage(user_used_node,node_top_destination,50,hour_node_top_num,50)
computer_coverage(user_used_node,node_top_destination,50000000,hour_node_top_num,50000000)

0.7376414966833004
0.7829043770800463
0.8521930898633576
0.8974265301179345
0.9149501056499647
0.9234556303602804
0.9280068087019707
0.9350483554335127


In [ ]:
total_node = 0
for user in user_used_node:
    total_node+=len(user_used_node[user])
print(total_node/len(user_used_node))
total_node = 0
for start in node_top_destination:
    total_node+=len(node_top_destination[start])
print(total_node/len(node_top_destination))

In [ ]:
computer_coverage(user_used_node,node_top_destination,3)
computer_coverage(user_used_node,node_top_destination,5)
computer_coverage(user_used_node,node_top_destination,10)
computer_coverage(user_used_node,node_top_destination,20)
computer_coverage(user_used_node,node_top_destination,30)
computer_coverage(user_used_node,node_top_destination,40)
computer_coverage(user_used_node,node_top_destination,50)
computer_coverage(user_used_node,node_top_destination,50000000)

In [ ]:
computer_coverage(user_used_node,node_top_destination,3)
computer_coverage(user_used_node,node_top_destination,5)
computer_coverage(user_used_node,node_top_destination,10)
computer_coverage(user_used_node,node_top_destination,20)
computer_coverage(user_used_node,node_top_destination,30)
computer_coverage(user_used_node,node_top_destination,40)
computer_coverage(user_used_node,node_top_destination,50)
computer_coverage(user_used_node,node_top_destination,50000000)

In [ ]:
total_node = 0
for user in user_used_node:
    total_node+=len(user_used_node[user])
print(total_node/len(user_used_node))
total_node = 0
for start in node_top_destination:
    total_node+=len(node_top_destination[start])
print(total_node/len(node_top_destination))

In [ ]:
with open('./data/temp/Candidate/user_used_node','wb+') as f:
    pickle.dump(user_used_node, f, pickle.HIGHEST_PROTOCOL)

